In [ ]:
from filterpy.kalman import KalmanFilter

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import pymc as pm
import arviz as az
from retry import retry
from tqdm import tqdm
from fastprogress import fastprogress
fastprogress.printing = lambda: True

In [ ]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats, playergamelog
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd
import time

**Get player 3 point shooting data**

In [ ]:
@retry()
def get_player_game_percentages(player_id):
  all_seasons = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]["SEASON_ID"].tolist()
  logs = []
  for i in all_seasons:
    logs.append(playergamelog.PlayerGameLog(player_id=player_id, season=i).get_data_frames()[0])
    time.sleep(.25)

  log = pd.concat(logs)
  log = log.sort_values('Game_ID')

  attempts = log["FG3A"]
  makes = log["FG3M"]
  total_attempts = attempts.sum()
  total_makes = makes.sum()
  career_percentage = total_makes / total_attempts

  return attempts.tolist(), makes.tolist(), career_percentage

In [ ]:
def convert_to_binary_array(attempts, makes):
    arr = []
    for attempt, make in zip(attempts, makes):
        arr.extend([1] * make)
        arr.extend([0] * (attempt - make))
    return arr

In [ ]:
def b3P(player_id):
  shooting_attempts, shooting_successes, perc = get_player_game_percentages(player_id)
  shots = convert_to_binary_array(shooting_attempts, shooting_successes)

  with pm.Model() as model:

      theta = pm.Beta('theta', alpha=1, beta=1)
      y = pm.Bernoulli('y', p=theta, observed=shots)

      start = pm.find_MAP()
      step = pm.Metropolis()
      trace = pm.sample(len(shots), step=step, start=start)
      summary_stats = az.summary(trace, var_names=['theta'], hdi_prob=0.95)
      mean = summary_stats.loc['theta', 'mean']
      hdi_lower = summary_stats.loc['theta', 'hdi_2.5%']
      hdi_upper = summary_stats.loc['theta', 'hdi_97.5%']

  #return mean_shooting_skill, hdi_lower, hdi_upper

  return trace, mean, hdi_lower, hdi_upper

In [ ]:
def b3P_arr(player_id, stepsize):
  shooting_attempts, shooting_successes, perc = get_player_game_percentages(player_id)
  shots = convert_to_binary_array(shooting_attempts, shooting_successes)
  mean_arr = []
  hdi_lower_arr = []
  hdi_upper_arr = []
  stepsize = int(len(shots) / 100)
  for i in tqdm(range(0, len(shots), stepsize)):
    with pm.Model() as model:
      upper_ind = int(max(len(shots), i + stepsize))
      subset_shots = shots[0:upper_ind]
      theta = pm.Beta('theta', alpha=1, beta=1)
      y = pm.Bernoulli('y', p=theta, observed=subset_shots)

      start = pm.find_MAP()
      step = pm.Metropolis()
      trace = pm.sample(len(subset_shots), step=step, start=start)
      summary_stats = az.summary(trace, var_names=['theta'], hdi_prob=0.95)
      mean = summary_stats.loc['theta', 'mean']
      hdi_lower = summary_stats.loc['theta', 'hdi_2.5%']
      hdi_upper = summary_stats.loc['theta', 'hdi_97.5%']

    mean_arr.append(mean)
    hdi_lower_arr.append(hdi_lower)
    hdi_upper_arr.append(hdi_upper)

  return mean_arr, hdi_lower_arr, hdi_upper_arr

In [ ]:
def kalman(mean, burnin):
  kf = KalmanFilter(dim_x=1, dim_z=1)
  kf.F = np.array([[1.]])
  kf.H = np.array([[1.]])

  q_var = np.var(mean[:burnin])
  initial_r_var = q_var
  kf.Q = np.array([[q_var]])
  kf.R = np.array([[q_var]])

  x_rest = list()
  kf.x = np.array([0])
  kf.P *= 10.
  test = mean[burnin:]
  decay_rate = .01
  for i, y in enumerate(test[:-1]):
    kf.predict()

    #Weighting
    weight = np.exp(decay_rate * i)
    kf.R = np.array([[initial_r_var / weight]])

    x_rest.append(kf.x[0])
    kf.update([y])

  return [0] + list(mean[:burnin]) + x_rest



In [ ]:
def kBP(player_id, stepsize=100, burnin=10):
  mean, hdi_lower, hdi_upper = b3P_arr(player_id, stepsize)
  xs = kalman(mean, burnin)
  return xs[-1]

In [ ]:
xs, hdi_lower, hdi_upper = kBP(1629636)

In [ ]:
xs[-1]

0.3838061488907534

In [ ]:
xs

[0,
 0.384,
 0.384,
 0.385,
 0.383,
 0.384,
 0.385,
 0.384,
 0.385,
 0.384,
 0.384,
 0.0,
 0.383999986176001,
 0.38399999542266894,
 0.38399999830205656,
 0.3846248817342444,
 0.38423386525985403,
 0.38408715568519686,
 0.38466134087750237,
 0.384874933689044,
 0.3849540252692474,
 0.38434908551129227,
 0.38412714196653125,
 0.3840460917574501,
 0.3840166313281848,
 0.38400597299276007,
 0.38400213506197317,
 0.3840007595816514,
 0.3840002689527286,
 0.3840000947778751,
 0.3846493196955398,
 0.3842266337730107,
 0.38473136861818324,
 0.3849071406273674,
 0.3843120563504994,
 0.3847644990721901,
 0.38426042939244004,
 0.38474929795608914,
 0.3842527492546379,
 0.3840848348349026,
 0.3840283334487312,
 0.3840094158042456,
 0.3840031134366048,
 0.3853430194981119,
 0.38443963243718793,
 0.3841431849256944,
 0.38404639774292604,
 0.3840149582301962,
 0.3840047977673052,
 0.3846824312797478,
 0.3842166429205242,
 0.3847526019162478,
 0.38423645289450736,
 0.3840739023880797,
 0.384022977278